In [7]:
# A decision tree to predict wether an animal is a cat or not a cat
# Input: some body info
# Output: 1 (cat) or 0 (not cat)

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [10]:
# LOADING DATA FROM CVS FILE

dataset = pd.read_csv('trainingSet_decisionTree.csv')
dataset.head()

,Ear Shape,Face Shape,Whiskers,Cat
0,Pointy,Round,Present,1
1,Floppy,Not Round,Present,1
2,Floppy,Round,Absent,0
3,Pointy,Not Round,Present,0
4,Pointy,Round,Present,1


In [11]:
# one_hot encode
dataset_onehot = pd.get_dummies(dataset, columns=['Ear Shape', 'Face Shape', 'Whiskers'], dtype=int)
dataset_onehot.head()

,Cat,Ear Shape_Floppy,Ear Shape_Pointy,Face Shape_Not Round,Face Shape_Round,Whiskers_Absent,Whiskers_Present
0,1,0,1,0,1,0,1
1,1,1,0,1,0,0,1
2,0,1,0,0,1,1,0
3,0,0,1,1,0,0,1
4,1,0,1,0,1,0,1


In [54]:
input = dataset_onehot[['Ear Shape_Floppy', 'Face Shape_Round', 'Whiskers_Present']].to_numpy()
print(input, end='\n\n')

output = dataset_onehot['Cat'].to_numpy()
# output = dataset_onehot.Cat.to_numpy() # other way to do the same
print(output)

[[0 1 1]
 [1 0 1]
 [1 1 0]
 [0 0 1]
 [0 1 1]
 [0 1 0]
 [1 0 0]
 [0 1 0]
 [1 1 0]
 [1 1 0]]

[1 1 0 0 1 1 0 1 0 0]


To measure the **impurity** of a node it's used the entropy equation ($H$), defined as

$$H(p_1) = -p_1 \text{log}_2(p_1) - (1- p_1) \text{log}_2(1- p_1)$$

In a decision tree, to decide if a node will be split or not we look at the **information gain** that split would give us:

$$\text{Information Gain} = H(p_1^\text{node})- \left(w^{\text{left}}H\left(p_1^\text{left}\right) + w^{\text{right}}H\left(p_1^\text{right}\right)\right)$$

In [47]:
class Node:
    # value (prediction class value) is None if self isn't a leaf node
    # feature (feature used to split) is None if self is a leaf node
    def __init__(self, value=None, feature=None, right=None, left=None):
        self.feature = feature
        self.value = value
        self.right = right # feature == 0
        self.left = left # feature == 1

    def is_leaf(self):
        return (self.value != None)

class Desicion_tree:
    def __init__(self, min_samples = 2, max_depth = 10, info_gain_threshold = 0.25):
        self.min_samples = min_samples
        self.max_depth = max_depth
        self.info_gain_thr = info_gain_threshold
        self.root = None
    
    def fit(self, X, y):
        self.root = self._build_tree(X, y)
            

    def _build_tree(self, X, y, depth = 0):
        labels_one = sum(y)
        total_samples = len(y)
        
        if (self.max_depth <= depth) or (self.min_samples > total_samples) or labels_one == total_samples or labels_one == 0:
            return Node(value = 1 if (labels_one > int(total_samples/2)) else 0)
        
        left_idxs, right_idxs, feature_idx, info_gain = self._best_split(X, y)
        if info_gain < self.info_gain_thr:
            return Node(value = 1 if (labels_one > int(total_samples/2)) else 0)
            
        left_tree = self._build_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right_tree = self._build_tree(X[right_idxs, :], y[right_idxs], depth+1)

        return Node(feature=feature_idx, right=right_tree, left=left_tree)

    def _best_split(self, X, y):
        best_info_gain = -1
        feature_idx = 0

        features = X.shape[1]
        for f in range(features):
            left_idxs, right_idxs = self._split(X, f)
            curr_info_gain = self._info_gain(X, y, left_idxs, right_idxs)
            if curr_info_gain > best_info_gain:
                best_info_gain = curr_info_gain
                feature_idx = f
        
        left, right = self._split(X, feature_idx)
        return left, right, feature_idx, best_info_gain

    def _split(self, x, feature_index):
        # left_indices: feature == 1
        # right_indices: feature == 0
    
        left_indices = []
        right_indices = []
        
        rows = x.shape[0]
        for i in range(rows):
            if x[i, feature_index] == 1:
                left_indices.append(i)
            else:
                right_indices.append(i)
    
        return left_indices, right_indices

    def _info_gain(self, x, y, left_indices, right_indices):
        p = sum(y) / len(y)
        curr_entropy = self._entropy(p)
    
        return curr_entropy - self._weighted_entropy(x, y, left_indices, right_indices)
    
    def _weighted_entropy(self, x, y, left_indices, right_indices):
        num_data_left = len(left_indices)
        num_data_right = len(right_indices)
        
        p_left = 0 if num_data_left == 0 else sum(y[left_indices]) / num_data_left # fraction of class 1 in the left node
        p_right = 0 if num_data_right == 0 else sum(y[right_indices]) / num_data_right # fraction of class 1 in the right node
    
        return (num_data_left*self._entropy(p_left) + num_data_right*self._entropy(p_right)) / len(y)

    def _entropy(self, p):
        if p == 0 or p == 1:
            return 0
        return -1 * (p*np.log2(p) + (1-p)*np.log2(1-p))

    def predict(self, X):
        prediction = np.zeros(X.shape[0])
        y = []
        for x in X:
            y.append(self._leaf_value(x, self.root))

        return y

    def _leaf_value(self, x, node):
        if node.is_leaf():
            return node.value
        if x[node.feature] == 1:
            return self._leaf_value(x, node.left)
        return self._leaf_value(x, node.right)
            
        


In [48]:
d_tree = Desicion_tree()

In [49]:
d_tree.fit(input, output)

In [56]:
print(d_tree.predict(input))
print(list(output))

[1, 1, 0, 0, 1, 1, 0, 1, 0, 0]
[1, 1, 0, 0, 1, 1, 0, 1, 0, 0]
